# CSV TRANSFORMER

Creator: Riley Cohen 

**Background:**

Being a data driven company, Insellerate often migrated clients lead data into the Insellerate DataBase. The issue was the the data had to be in a specific format for the upload to be successful. The format consisted of a set of column or field names as well as a specific set of field values for a subset of the field names called "drop down fields", which contain "drop down values. The drop down fields are essentially a subset of the insellerate fields that have a fixed set of values. Insellerate originally did the migration process in Excel, but it was an extremely slow and tedious process. As a short term and better solution to this problem, I created this notebook that makes the process far less tedious and time consuming. I used this notebook extensively during my internship and used to migrate over 30 datasets.
_________________________________________________________________________________________________________________
**General Process**

In this project I utilized Jupyter's widgets to creat a User Interfaced used to tranform csv data. The interface does the following:

1. Prompts the user to match the input field names to the corresponding Insellerate field names
2. Prompts the user to match the their drop down values to the corresponding Insellerate drop down values
3. Transfrom the csv into the necessary Insellerate format

Note: To use this notebook, it is important that you run each cell one at a time. 
    

Github does not display the widgets, however I made a video demo on how to use this notebook. Click on the image below for a demo! :-)

[![](http://img.youtube.com/vi/xhKAYLUjKts/0.jpg)](http://www.youtube.com/watch?v=xhKAYLUjKts "title")



In [20]:
#Import necessary data
import pandas as pd #For data manipulation
from difflib import SequenceMatcher, get_close_matches #For predicitng field
import numpy as np
from ipywidgets import *
from ipywidgets import HTML, HBox
from ipywidgets import interact, interactive, fixed, interact_manual #For dropdown
import re 

# First Read the Data

Directions to read google doc data directly from url:

1. Share -> change -> Restricted -> Anyone with link

2. Get URL(s)

    url1 = https://docs.google.com/spreadsheets/d/1jsLciiiWB6HtxDVgr9WsbcbYdF8lMEyDPbRCDkyqu3Q/edit#gid=246503311

    url2 = https://docs.google.com/spreadsheets/d/1jsLciiiWB6HtxDVgr9WsbcbYdF8lMEyDPbRCDkyqu3Q/edit#gid=33278173
    

3. Extract sheet id and gid

    sheetid = '18NMIIkbovJud9py6KqDORYGeQX3eCC5UVD5Am56pKcE'
    
    GID = 246503311

4. Place data into the following link & read the csv

    input = https://docs.google.com/spreadsheets/d/sheetid/export?format=csv&gid=GID

    pd.read_csv(input)

In [21]:
def read_sheet(url):
    '''
    Takes in google sheet url and returns dataframe
    '''
    sheetid = url[39:83]
    gid = url.split(r'=')[1]
    link = 'https://docs.google.com/spreadsheets/d/'+sheetid+'/export?format=csv&gid='+gid
    data = pd.read_csv(link)
    return data

In [22]:
#Import client data
client_data = read_sheet('https://docs.google.com/spreadsheets/d/1jsLciiiWB6HtxDVgr9WsbcbYdF8lMEyDPbRCDkyqu3Q/edit#gid=33278173')
client_headers  = list(client_data.columns)

#Import Insellerate field names
Insellerate_Fields = pd.read_csv('APIDs.csv')['APID'].to_frame() #Insellerate database headers
Insellerate_Headers = list(Insellerate_Fields['APID'])

#Import Insellerate Drop Down Values
link = 'https://docs.google.com/spreadsheets/d/1jsLciiiWB6HtxDVgr9WsbcbYdF8lMEyDPbRCDkyqu3Q/edit#gid=246503311'
drop_downs_df = read_sheet(link)
drop_down_names = list(drop_downs_df.columns)
#Create dictory of drop down field and their values
temp = drop_downs_df.to_dict('list')
insell_dd_map = {k:[elem for elem in v if elem is not np.nan] for k,v in temp.items()}
insell_dd_map


display(client_data.head())
display(Insellerate_Fields.head())
display(drop_downs_df.head())







,Reference ID,Borrower First Name,Last Name,Email,Phone,Residence Address,Property Address,Property City,Property State,Zip Code,Residence,Property Kind
0,1066282,Isaac,Rosa,IsaacRosa@email.com,9.494313e+09,"2301 W. Big Beaver Rd, Ste. 525",2502 Rocky Point Drive,NaN,GA,33607.0,First,A
1,1066286,Amanda,Kotcher,AmandaKotcher@email.com,9.493151e+09,"750 Prides Crossing Suite 303 Newark, DE 19713",761 Old Hickory Blvd. Ste 400,NaN,OH,37027.0,First,Condo
2,1066294,Anissa,Shafranek,AnissaShafranek@email.com,9.493151e+09,"1615 5th Ave San Rafael, CA 94901",1525 South Belt Line Road,NaN,NaN,75019.0,Third,A
3,1066309,Elena,Sitarz,ElenaSitarz@email.com,7.759706e+09,"375 Northridge Road Suite 600 Atlanta, GA 30350",25 Enterprise Center,NaN,CA,2842.0,Third,Condo
4,1066313,Marcy,Gates,MarcyGates@email.com,9.493151e+09,"18301 Von Karman Suite 350 Irvine, CA 92612",324 S. Kalmia Street,NaN,CA,92025.0,Second,Condo


,APID
0,Campaign ID
1,Reference ID
2,LOS File ID
3,Existing Mortgage
4,Manager Reviewed


,Residence Type,Property Type,Loan Rate Type,Loan Type,PropertyCondition
0,Primary,Attached,Fixed Rate,Veterans Affairs,Not Specified
1,Secondary,Detached,Graduated Payment Mortgage,Federal Housing Administration,Good
2,Investment,Condominium,Growing-Equity Mortgage,Conventional,Fair
3,NaN,Co-Operative,Adjustable Rate Mortgage,USDA/Rural,Poor
4,NaN,Manufactured Housing,Other,Other,NaN


# Define useful functions

In [23]:
def autoFill(opts=[''], val='',txt='',placehold='Please type here to search...',callback=False):
    '''
    This function generates a keyword search widget
    '''
    opts.append('')
    def dropFunc(value):
        if (value.new in opts):
            dropClose()
            if (callable(callback)):
                callback(value) 
        text.value = value.new                
    def dropClose():
        drop.layout.visibility='hidden'
        selDropBox.layout.visibility='hidden'            
        selDropBox.layout.display='none'         
    def textFunc(value):
        matched = False
        if (len(value.new)>len(value.old)):
            if (len(value.new)>2):
                word = value.new
                out = [word]
                for mystring in opts:
                    if word.lower() in mystring.lower(): 
                        if (mystring.lower()==word.lower()):
                            matched = True
                        out.append(mystring)
                if (not matched):
                    drop.layout.visibility='visible'
                    selDropBox.layout.visibility='visible'
                    selDropBox.layout.display='flex'
                    out.append('')
                    drop.options=out 
                else:
                    dropClose()                    
        
    drop = Select(
                options=opts,
                value=val,
                rows=10,
                description=txt,
                disabled=False,
           )     
    text = Text(
                value=val,
                placeholder=placehold,
                description=txt,
                disabled=False,    
            )         
    drop.observe(dropFunc, names='value')
    text.observe (textFunc,names='value')
    selTextBox = Box([text])
    selDropBox = Box([drop], layout = Layout(display='none', top='-32px', visibility='hidden', flex_flow='column'))
    return (VBox([selTextBox, selDropBox],layout = Layout(display='flex', flex_flow='column'))) 


def open(value):
    show.value=value.new
    
show=HTML('')


def get_macthes(client_headers, Insellerate_Headers):
    '''
    This function takes in two lists client_headers, Insellerate_Headers, which contain the clients csv field names
    and insellerate csv field names respectively.
    
    It outputs a dictionary in which the keys are the clients field names and the values are possible matches 
    from the insellerate field names.
    '''

    client_headers, Insellerate_Headers = set(client_headers), set(Insellerate_Headers) #Turn lists into sets
    in_insellerate = client_headers.intersection(Insellerate_Headers) #Fields name that have a direct match
    need_match = client_headers - in_insellerate #Field names that don't have a direct match

    direct_matches = {k : k for k in in_insellerate}
    matches = {k : get_close_matches_icase(k, Insellerate_Headers, n=5, cutoff=.05) for k in need_match}
    mappings = matches.copy()
    mappings.update(direct_matches)
    
    return matches

def get_close_matches_icase(word, possibilities, *args, **kwargs):
    """ Case-insensitive version of difflib.get_close_matches """
    lword = word.lower()
    lpos = {p.lower(): p for p in possibilities}
    lmatches = get_close_matches(lword, lpos.keys(), n=5,cutoff=.05)
    return [lpos[m] for m in lmatches]

def open(value):
    show.value=value.new
    
show=HTML('')





def ui_header():
    
    '''
    Creates the header of the field mapping UI
    '''
    
    a = widgets.Text(value = 'Field Name' , disabled = False, layout={'width': '150px'})
    b = widgets.Text(value = 'Proposed Insellerate Name(s)', disabled = False, layout={'width': '260px'})
    c = widgets.Text(value = 'Search', disabled = False, layout={'width': '260px'})
    d = widgets.Text(value = 'Insellerate Fields', disabled = False, layout={'width': '300px'})
    
    input_widgets = widgets.HBox([a, b ,c ,d])
    display(input_widgets)
    
    return None



def ui_row(field_name, matches):
    
    '''
    Generates a row of the field mapping UI
    Takes in client field names and close matching insellerate field names returned by get_close_matches
    '''
    
    #Stores the users choices 
    chosen_proposed_values = [] #will hold user chosen dropdowns from proposed mapping
    chosen_insell_values = [] #will hold user chosen dropdown from insellerate headers
    chosen_search_values = [] #will hold user command search 


    #Create the UI
    clients_field_name = widgets.ToggleButtons(options=[field_name])
    proposed_value = widgets.Dropdown(options = matches, value = None, layout={'width': '300px'}) #Dropdown of proposed values
    insell_value = widgets.Dropdown(options = Insellerate_Headers, value = None, layout={'width': '300px'}) #Dropdown of insellerate headers
    search = autoFill(Insellerate_Headers, callback=open)
    
    chosen_proposed_values.append(proposed_value) #Keeps track of chosen values
    chosen_insell_values.append(insell_value) #Keeps track of chosen values
    chosen_search_values.append(search)    
    
    display(widgets.HBox([clients_field_name, 
                          proposed_value, 
                          search, 
                          insell_value]))
    
    return np.array([chosen_proposed_values, chosen_search_values, chosen_insell_values]).T

def field_map(keys, values):
    '''
    Takes in keys and values from close matches produced by get_close_matches
    Generates field mapping UI and returns an array of the widgets
    '''
    storage = tuple((ui_row(keys[k], values[k]) for k in range(len(keys))))
    return np.vstack(storage)


def get_names(z):
    
    '''
    Takes in an array of widgets
    Return an list consisting off all the values of the widgets
    '''
    
    def value(widget):
        return widget.value #Extract dropdown widget value

    def children(widget):
        return widget.children[0].children[0].value #Extract search widget value
    
    length = z.shape[0]
    z[:,0] = np.array([value(z[k,0]) for k in range(length)])
    z[:,1] = np.array([children(z[k,1]) for k in range(length)])
    z[:,2] = np.array([value(z[k,2]) for k in range(length)])
    
    z = pd.DataFrame(z).replace('','*').fillna('*').sum(axis = 1).str.replace('*','')
    
    return list(z)



def drop_down_map(chosen_names):
    
    '''
    Takes in list of names chose by field mapping
    Generates value mapping UI and returns array of widgets containing the value mappings
    '''
    
    #Names of the drop downs from previously chosen fields
    is_drop_down = [name in drop_down_names for name in chosen_names] #True if name is drop down
    index = [i for i, x in enumerate(is_drop_down) if x] #Indices for true values
    names = np.array(np.array(chosen_names))[index] #Names of just the drop downs
    
    #Matching drop down insellerate mappings
    dd_to_map = {k:insell_dd_map.get(k) for k in names}
    
    #Clients drop down mappigns
    names = [reverse_maps.get(name) for name in names]
    temp = client_data[names].to_dict('list')
    client_dd_map = {k:list(set([elem for elem in v if elem is not np.nan])) for k,v in temp.items()}
    
    keys = list(dd_to_map.keys()) #Field names
    client_values = list(client_dd_map.values()) #Clients fild values 
    insell_values = list(dd_to_map.values()) #Insellerate field values
    
    #Start the UI
    tab_nest = widgets.Tab() #Initialize the tabs
    a = widgets.Text(value = 'Field Value' , disabled = False, layout={'width': '150px'})
    d = widgets.Text(value = 'Insellerate Values', disabled = False, layout={'width': '270px'})
    header = widgets.HBox([a, d])

    children = [] #Will hold the children of the tab
    store = {} #Stores mapping choses values for each tab, will be a list of lists

    for c in range(len(keys)):

        UI = [header] #header is row1 of tab c


        for k in range(len(client_values[c])):

            #Create a row of the UI
            d1 = widgets.ToggleButtons(options = [client_values[c][k]]) #row k column 1 of tab c
            d2 = widgets.Dropdown(options = insell_values[c], value = None) # row k column 2 of tab c
            temp = [d1, d2]
            row = widgets.HBox(temp)
            UI.append(row)
            
            #store mapping
            store.update({temp[0]:temp[1]})

        
        children.append(widgets.VBox(UI))    
        tab_nest.set_title(c, keys[c])

    tab_nest.children = children #Set children
    display(tab_nest)  #display UI
    
    return store

# Step 1: Find Insellerate column names that are a close match to the clients column names and then generate the field mapping UI

In [24]:
#Close matches
mapping = get_macthes(client_headers, Insellerate_Headers)
keys = list(mapping.keys())
values = list(mapping.values())

#Generate UI
ui_header()
widget_array = field_map(keys, values)

# Step 2: Get the names of the needed field mapping

In [25]:
#Get names from 
chosen_names = get_names(widget_array.copy())

#Define mappings
maps = {keys[k]:chosen_names[k] for k in range(len(keys))} #client ->insell
reverse_maps = {val:key for key, val in maps.items()} #insell->client

store = drop_down_map(chosen_names)

# Step 3: Create the field value mappings

In [26]:
#Create the value mappings
value_mapping = {k.value:v.value for k,v in store.items()}

# Step 4: Transform the CSV into the insellerate Database format 

In [27]:
#Transform the csv
df = client_data.copy()
df = df.rename(columns = maps)
df = df.applymap(lambda x : value_mapping.get(x) if x in list(value_mapping.keys()) else x)
df.head()

,Reference ID,First Name,Last Name,Email,Mobile Phone,Mailing Address 1,Property Address 1,Property City,Property State,Property Zip Code,Residence Type,Property Type
0,1066282,Isaac,Rosa,IsaacRosa@email.com,9.494313e+09,"2301 W. Big Beaver Rd, Ste. 525",2502 Rocky Point Drive,NaN,GA,33607.0,Primary,Attached
1,1066286,Amanda,Kotcher,AmandaKotcher@email.com,9.493151e+09,"750 Prides Crossing Suite 303 Newark, DE 19713",761 Old Hickory Blvd. Ste 400,NaN,OH,37027.0,Primary,Condominium
2,1066294,Anissa,Shafranek,AnissaShafranek@email.com,9.493151e+09,"1615 5th Ave San Rafael, CA 94901",1525 South Belt Line Road,NaN,NaN,75019.0,Investment,Attached
3,1066309,Elena,Sitarz,ElenaSitarz@email.com,7.759706e+09,"375 Northridge Road Suite 600 Atlanta, GA 30350",25 Enterprise Center,NaN,CA,2842.0,Investment,Condominium
4,1066313,Marcy,Gates,MarcyGates@email.com,9.493151e+09,"18301 Von Karman Suite 350 Irvine, CA 92612",324 S. Kalmia Street,NaN,CA,92025.0,Secondary,Condominium


In [28]:
client_data.head()

,Reference ID,Borrower First Name,Last Name,Email,Phone,Residence Address,Property Address,Property City,Property State,Zip Code,Residence,Property Kind
0,1066282,Isaac,Rosa,IsaacRosa@email.com,9.494313e+09,"2301 W. Big Beaver Rd, Ste. 525",2502 Rocky Point Drive,NaN,GA,33607.0,First,A
1,1066286,Amanda,Kotcher,AmandaKotcher@email.com,9.493151e+09,"750 Prides Crossing Suite 303 Newark, DE 19713",761 Old Hickory Blvd. Ste 400,NaN,OH,37027.0,First,Condo
2,1066294,Anissa,Shafranek,AnissaShafranek@email.com,9.493151e+09,"1615 5th Ave San Rafael, CA 94901",1525 South Belt Line Road,NaN,NaN,75019.0,Third,A
3,1066309,Elena,Sitarz,ElenaSitarz@email.com,7.759706e+09,"375 Northridge Road Suite 600 Atlanta, GA 30350",25 Enterprise Center,NaN,CA,2842.0,Third,Condo
4,1066313,Marcy,Gates,MarcyGates@email.com,9.493151e+09,"18301 Von Karman Suite 350 Irvine, CA 92612",324 S. Kalmia Street,NaN,CA,92025.0,Second,Condo
